In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms,models
import torch.multiprocessing as mp
import argparse
from torch import optim
import os
import visdom

In [12]:
alexNet  = models.AlexNet()
alexNet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

### dropout

In [11]:
a = torch.Tensor([[1.2,2.3,3.4],[6.7,7.8,1.09]])
b = nn.Dropout(0.5)
b(a)

tensor([[ 0.0000,  4.6000,  0.0000],
        [13.4000, 15.6000,  2.1800]])

### ZF-Net

In [19]:
class ZFNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,96,kernel_size=(7,7),stride=(2,2)),
            nn.MaxPool2d((3,3),stride=(2,2)),
            nn.Conv2d(96,256,kernel_size=(5,5),stride=(2,2)),
            nn.MaxPool2d((3,3),stride=(2,2)),
            nn.Conv2d(256,384,kernel_size=(3,3),stride=(1,1)),
            nn.Conv2d(384,384,kernel_size=(3,3),stride=(1,1)),
            nn.Conv2d(384,256,kernel_size=(3,3),stride=(1,1)),
            nn.MaxPool2d((3,3),stride=(2,2))
        )
        self.classifier = nn.Sequential(
            nn.Linear(43264,4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,1000)
        )
    def forward(self,x):
        x = self.features(x)
        x = x.view(-1,43264)
        x = self.classifier(x)
        return x

In [20]:
zf_net = ZFNet()

In [21]:
zf_net

ZFNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2))
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
    (5): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))
    (6): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1))
    (7): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=43264, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)

In [22]:
### VGG-16

In [24]:
VGG16 = models.vgg16()
VGG16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

### Inception结构

In [58]:
import torch.nn as nn
import torch
class BasicConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,**kwargs):
        super(BasicConv2d,self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,bias=False,**kwargs)
        self.bn = nn.BatchNorm2d(out_channels,eps=0.001)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x,inplace=True)

class Inception(nn.Module):
    def __init__(self,in_channels,pool_features):
        super(Inception,self).__init__()
        self.branch1X1 = BasicConv2d(in_channels,64,kernel_size = 1)

        self.branch5X5_1 = BasicConv2d(in_channels,48,kernel_size = 1)
        self.branch5X5_2 = BasicConv2d(48,64,kernel_size=5,padding = 2)

        self.branch3X3_1 = BasicConv2d(in_channels,64,kernel_size = 1)
        self.branch3X3_2 = BasicConv2d(64,96,kernel_size = 3,padding = 1)

        self.branch_pool = BasicConv2d(in_channels,pool_features,kernel_size = 1)
    def forward(self, x):
        branch1X1 = self.branch1X1(x)

        branch5X5 = self.branch5X5_1(x)
        branch5X5 = self.branch5X5_2(branch5X5)

        branch3X3 = self.branch3X3_1(x)
        branch3X3 = self.branch3X3_2(branch3X3)

        branch_pool = F.avg_pool2d(x,kernel_size = 3,stride = 1,padding = 1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1X1,branch3X3,branch5X5,branch_pool]
        return torch.cat(outputs,1)

In [51]:
a = models.inception

In [54]:
dir(a)

['BasicConv2d',
 'F',
 'Inception3',
 'InceptionA',
 'InceptionAux',
 'InceptionB',
 'InceptionC',
 'InceptionD',
 'InceptionE',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'inception_v3',
 'model_urls',
 'model_zoo',
 'nn',
 'torch']

### ResNet

In [84]:
list(filter(lambda x: x.startswith("r") or x.startswith("B"),dir(models.resnet)))

['BasicBlock',
 'Bottleneck',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50']